In [48]:
# !pip install opencv-python
# !pip install mediapipe

In [1]:
import cv2
import time
import mediapipe as mp

2023-07-23 19:58:15.418527: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-23 19:58:15.498381: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-23 19:58:15.831425: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-23 19:58:15.832584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-23 19:58:16.671704: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [5]:
import cv2
import mediapipe as mp
import time
import numpy as np
import random

def calculate_angle_between_2_joints(frame_width, frame_height, landmarks, a, b, c, inner_angle=True):
    # calculates the angle between 3 vectors a, b, c, where b is the vertex

    # Get the coordinates of the left shoulder and hip
    a = landmarks.landmark[a]
    b = landmarks.landmark[b]
    c = landmarks.landmark[c]


    # unnormalized pixel values
    a_x, a_y = a.x * frame_width, a.y * frame_height
    b_x, b_y = b.x * frame_width, b.y * frame_height
    c_x, c_y = c.x * frame_width, c.y * frame_height

    # angle <abc
    angle_rad = np.arctan2(c_y - b_y, c_x - b_x) - np.arctan2(a_y - b_y, a_x - b_x)
    angle_deg = np.abs(angle_rad * 180.0 / np.pi).astype(int)

    if inner_angle:
        return angle_deg
    else:
        return 360 - angle_deg

def calculate_all_joint_angles(frame_width, frame_height,cyclist_is_facing_right, landmarks, mpPose):
    angles = {
        "hip_angle": None,
        "knee_angle": None,
        "ankling_range": None,
        "shoulder_angle": None,
        "elbow_angle": None
    }

    # ideal angles for a person facing left on a racing bike
    ideal_angles = {
        "hip_angle": {"min": 60, "max": 110},
        "knee_angle": {"min": 65, "max": 145},
        "ankling_range": {"min": 115, "max": 180},
        "shoulder_angle": {"min": 65, "max": 75},
        "elbow_angle": {"min": 150, "max": 160}
    }

    if cyclist_is_facing_right:
        angles["hip_angle"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.RIGHT_SHOULDER, mpPose.PoseLandmark.RIGHT_HIP, mpPose.PoseLandmark.RIGHT_KNEE)
        angles["knee_angle"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.RIGHT_HIP, mpPose.PoseLandmark.RIGHT_KNEE, mpPose.PoseLandmark.RIGHT_ANKLE, inner_angle=False)
        angles["ankling_range"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.RIGHT_ANKLE, mpPose.PoseLandmark.RIGHT_HEEL, mpPose.PoseLandmark.RIGHT_FOOT_INDEX, inner_angle=False)
        angles["shoulder_angle"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.RIGHT_ELBOW, mpPose.PoseLandmark.RIGHT_SHOULDER, mpPose.PoseLandmark.RIGHT_HIP)
        angles["elbow_angle"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.RIGHT_SHOULDER, mpPose.PoseLandmark.RIGHT_ELBOW, mpPose.PoseLandmark.RIGHT_WRIST)
    else:
        angles["hip_angle"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.LEFT_SHOULDER, mpPose.PoseLandmark.LEFT_HIP, mpPose.PoseLandmark.LEFT_KNEE, inner_angle=False)
        angles["knee_angle"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.LEFT_HIP, mpPose.PoseLandmark.LEFT_KNEE, mpPose.PoseLandmark.LEFT_ANKLE)
        angles["ankling_range"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.LEFT_KNEE, mpPose.PoseLandmark.LEFT_ANKLE, mpPose.PoseLandmark.LEFT_HEEL)
        angles["shoulder_angle"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.LEFT_ELBOW, mpPose.PoseLandmark.LEFT_SHOULDER, mpPose.PoseLandmark.LEFT_HIP)
        angles["elbow_angle"] = calculate_angle_between_2_joints(frame_width, frame_height, landmarks, mpPose.PoseLandmark.LEFT_SHOULDER, mpPose.PoseLandmark.LEFT_ELBOW, mpPose.PoseLandmark.LEFT_WRIST, inner_angle=False)


    return angles


def run_pose_estimation(video_path):
    mpDraw = mp.solutions.drawing_utils
    mpPose = mp.solutions.pose
    pose = mpPose.Pose()

    cap = cv2.VideoCapture(video_path)

    frame_width, frame_height = int(cap.get(3)), int(cap.get(4))

    is_first_frame = True

    font= cv2.FONT_HERSHEY_SIMPLEX
    font_size = 1
    font_color = (255, 0, 0)
    table_height = 50

    while cap.isOpened():
        

        ret, frame = cap.read()
        if not ret:
            break
        pTime = 0

        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(imgRGB)

        if results.pose_landmarks:
            mpDraw.draw_landmarks(frame, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

            angles = calculate_all_joint_angles(frame_width, frame_height, True, results.pose_landmarks, mpPose)
        
            cv2.putText(frame, "hip angle (deg): " + str(angles["hip_angle"]), (50, 50), font, font_size, font_color, 2)
            cv2.putText(frame, "knee angle (deg): " + str(angles["knee_angle"]), (50, 50 + table_height), font, font_size, font_color, 2)
            cv2.putText(frame, "shoulder angle (deg): " + str(angles["shoulder_angle"]), (50, 50 + 2*table_height), font, font_size, font_color, 2)
            cv2.putText(frame, "elbow angle (deg): " + str(angles["elbow_angle"]), (50, 50 + 3*table_height), font, font_size, font_color, 2)
            cv2.putText(frame, "ankling range (deg): " + str(angles["ankling_range"] - 180), (50, 50 + 4*table_height), font, font_size, font_color, 2)

            # static text, todo: make it dynamic
            cv2.putText(frame, "posture:", (frame_width - 500, 50), font, font_size, font_color, 2)
            green = random.randint(200, 255)  # Random green value between 100 and 255
            red = random.randint(0, green-1)
            cv2.putText(frame, "good", (frame_width - 350, 50), font, font_size, (red,  green,  0), 2)


        cv2.imshow('frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


        # wait for 30s for the 1st frame
        if is_first_frame:
            time.sleep(30)
            is_first_frame = False

    cap.release()
    cv2.destroyAllWindows()


run_pose_estimation('./man-cycling.mp4')




QObject::moveToThread: Current thread (0x5628cba26de0) is not the object's thread (0x5628cba4f210).
Cannot move to target thread (0x5628cba26de0)

QObject::moveToThread: Current thread (0x5628cba26de0) is not the object's thread (0x5628cba4f210).
Cannot move to target thread (0x5628cba26de0)

QObject::moveToThread: Current thread (0x5628cba26de0) is not the object's thread (0x5628cba4f210).
Cannot move to target thread (0x5628cba26de0)

QObject::moveToThread: Current thread (0x5628cba26de0) is not the object's thread (0x5628cba4f210).
Cannot move to target thread (0x5628cba26de0)

QObject::moveToThread: Current thread (0x5628cba26de0) is not the object's thread (0x5628cba4f210).
Cannot move to target thread (0x5628cba26de0)

QObject::moveToThread: Current thread (0x5628cba26de0) is not the object's thread (0x5628cba4f210).
Cannot move to target thread (0x5628cba26de0)

QObject::moveToThread: Current thread (0x5628cba26de0) is not the object's thread (0x5628cba4f210).
Cannot move to tar

In [181]:
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose

img_url = './cycling-small.jpg'
show_image = True

# Initialize pose estimation
pose = mpPose.Pose()

# Read the image
# img = cv2.imread('./cycling-small.jpg')
img = cv2.imread(img_url)
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# flip the image
# imgRGB = cv2.flip(imgRGB, 1)
# img = cv2.flip(img, 1)


# Process the image
results = pose.process(imgRGB)
landmarks = results.pose_landmarks
# Draw pose landmarks on the image
if results.pose_landmarks:
    mpDraw.draw_landmarks(img, landmarks, mpPose.POSE_CONNECTIONS)

# Display the image
if show_image:
    cv2.imshow('img', img)

    # Wait for the user to press any key

    cv2.waitKey(0)
    cv2.destroyAllWindows()


# save the image
# cv2.imwrite('./cycling-small-landmarks.jpg', img)
save_url = img_url.split('.')[1] + '-landmarks.jpg'
cv2.imwrite(save_url, img)

QObject::moveToThread: Current thread (0x5592dcb55700) is not the object's thread (0x5592dcac4ab0).
Cannot move to target thread (0x5592dcb55700)

QObject::moveToThread: Current thread (0x5592dcb55700) is not the object's thread (0x5592dcac4ab0).
Cannot move to target thread (0x5592dcb55700)

QObject::moveToThread: Current thread (0x5592dcb55700) is not the object's thread (0x5592dcac4ab0).
Cannot move to target thread (0x5592dcb55700)

QObject::moveToThread: Current thread (0x5592dcb55700) is not the object's thread (0x5592dcac4ab0).
Cannot move to target thread (0x5592dcb55700)

QObject::moveToThread: Current thread (0x5592dcb55700) is not the object's thread (0x5592dcac4ab0).
Cannot move to target thread (0x5592dcb55700)

QObject::moveToThread: Current thread (0x5592dcb55700) is not the object's thread (0x5592dcac4ab0).
Cannot move to target thread (0x5592dcb55700)

QObject::moveToThread: Current thread (0x5592dcb55700) is not the object's thread (0x5592dcac4ab0).
Cannot move to tar

False

{'hip_angle': 122, 'knee_angle': 143, 'ankling_range': 152, 'shoulder_angle': 65, 'elbow_angle': 166}


In [187]:

for angle in angles:
    # angles[angle] = abs(angles[angle])

    if angles[angle] < ideal_angles[angle]["min"] or angles[angle] > ideal_angles[angle]["max"]:
        print(f"Your {angle} is not ideal. It should be between {ideal_angles[angle]['min']} and {ideal_angles[angle]['max']} degrees")
        
        # Adjust the bike components based on the calculated angles
        if angle == "hip_angle":
            # Adjust the seat height and fore/aft position to achieve the desired hip angle.
            # Lower the seat if the angle is too open, or raise it if the angle is too acute.
            print("To optimize your hip angle, add height to the seat if it is too low, or lower it if it is too high.")
            print("Additionally, adjust the seat position forward or backward to achieve the correct hip angle.")

        elif angle == "knee_angle":
            # Adjust the seat height to achieve the correct knee angle.
            # Lower the seat if the angle is too acute, and raise it if the angle is too open.
            print("To achieve the correct knee angle, adjust the seat height. Lower the seat if your knee angle is too acute, and raise it if it is too open.")

        elif angle == "ankling_range":
            # If using clipless pedals, adjust the cleats to achieve the desired ankle range.
            print("For optimizing your ankle range, adjust the cleats on your clipless pedals. Ensure that your feet are properly aligned with the pedals.")

        elif angle == "shoulder_angle":
            # Adjust the handlebar height and reach to achieve the desired shoulder angle.
            # Lower or raise the handlebars and adjust the stem length as needed.
            print("To optimize your shoulder angle, adjust the handlebar height and reach. Lower or raise the handlebars to achieve the correct shoulder angle.")
            print("You may also need to adjust the stem length to achieve a comfortable reach.")

        elif angle == "elbow_angle":
            # Further handlebar adjustments may be needed to achieve the correct elbow angle.
            # Raise or lower the handlebars and adjust their horizontal position if necessary.
            print("To achieve the correct elbow angle, fine-tune the handlebar position. Raise or lower the handlebars, and adjust their horizontal position as needed.")

        print("\n")
print(angles)

Your hip_angle is not ideal. It should be between 60 and 110 degrees
To optimize your hip angle, add height to the seat if it is too low, or lower it if it is too high.
Additionally, adjust the seat position forward or backward to achieve the correct hip angle.


Your elbow_angle is not ideal. It should be between 150 and 160 degrees
To achieve the correct elbow angle, fine-tune the handlebar position. Raise or lower the handlebars, and adjust their horizontal position as needed.


{'hip_angle': 122, 'knee_angle': 143, 'ankling_range': 152, 'shoulder_angle': 65, 'elbow_angle': 166}
